In [3]:
import hashlib
import json


class Block:
    def __init__(self, index, data, timestamp, previous_hash, nonce=0):
        self.index = index
        self.data = data
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

    def __repr__(self):
        return 'Block: {}'.format(self.__dict__)



In [26]:
import time
from timeit import default_timer as timer
from datetime import timedelta
import copy

class Blockchain:
    def __init__(self):
        self.new_data = []
        self.chain = []
        self.difficulty = 1
        self.integrity_lst=[]
        self.__create_genesis_block()

        
    def __create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), 0)
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    def add_new_data(self, data):
        self.new_data.append(data)

    def mine(self):
        if not self.new_data:
            return False
        last_block = self.last_block
        new_block = Block(index=last_block.index + 1,
                          data=self.new_data,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
        proof = self.__proof_of_work(new_block)
        self.__add_block(new_block, proof)
        self.new_data = []
        return new_block.index

    @property
    def last_block(self):
        return self.chain[-1]
    
    def __proof_of_work(self, block):
        start = timer()
        block.nonce = 0
        computed_hash = block.compute_hash()
        print('Mining block {}'.format(block.index))
        print('Difficulty: {}'.format(self.difficulty))
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        print('Computed hash: {}'.format(computed_hash))
        print('Nonce: {}'.format(block.nonce))
        end = timer()
        print('Elapsed time: {}'.format(timedelta(seconds=end - start)))
        return computed_hash

    def __is_valid_proof(self, block, block_hash):
        return block_hash.startswith('0' * self.difficulty) and block_hash == block.compute_hash()

    def __add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.__is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
    
    
    def verify_integrity(self):
        for i in self.chain:
            temp=copy.deepcopy(i)
            print(temp.__dict__)
            del temp.hash
            if temp.compute_hash()!=i.hash:
                print(temp.compute_hash())
                print(i.hash)
                print("Integrity Failed")
                print("Block {} got unverified change.".format(temp.index))
                return False
        for i in range(1,len(self.chain)-1):
            if self.chain[i].previous_hash!=self.chain[i-1].hash:
                print("Integrity Failed")
                return False
        print("Integrity Ensured")
        return True
    
    def view_by_hash_code(self, hash_code):
        for i in self.chain:
            if hash_code==i.hash:
                return i
        print("Nothing fit your search")
        
    def edit_by_hash_code(self, hash_code, new_data):
        delete_blocks=0
        for i in self.chain:
            if hash_code==i.hash:
                delete_blocks=i.index
                break
        print(delete_blocks)
        temp_chain=self.chain[delete_blocks+1:]
        self.chain=self.chain[:delete_blocks]
        self.add_new_data(new_data)
        print(temp_chain)
        print(self.chain)
        self.mine()
        for i in temp_chain:
            self.add_new_data(i.data)
            self.mine()

In [27]:
blockchain = Blockchain()
print(blockchain.chain)

[Block: {'index': 0, 'data': [], 'timestamp': 1668954286.3795156, 'previous_hash': 0, 'nonce': 0, 'hash': '9d1390a6cc109ba611856dbe989fa1172e81df25dda16c7bd049a4c21e405edf'}]


In [28]:
f = open('samples/1.json')
data = json.load(f)
blockchain.add_new_data(data)
blockchain.mine()

Mining block 1
Difficulty: 1
Computed hash: 0c7583ff72d2b76f7e9e51a8bd3881c647e1d87cb8cf641bc648eeb3676cb8f5
Nonce: 7
Elapsed time: 0:00:00.000541


1

In [29]:
f = open('samples/2.json')
data = json.load(f)
blockchain.add_new_data(data)
blockchain.mine()

Mining block 2
Difficulty: 1
Computed hash: 0b06b0e14ac3104e22caf02d404d126ba18008e6b20092992478523fb3e09a3a
Nonce: 24
Elapsed time: 0:00:00.001749


2

In [30]:
f = open('samples/3.json')
data = json.load(f)
blockchain.add_new_data(data)
blockchain.mine()

Mining block 3
Difficulty: 1
Computed hash: 061f82f849681bf0f993ce26547a89ff558984d0d878de4f977ba4dbaafa8ee7
Nonce: 0
Elapsed time: 0:00:00.000169


3

In [31]:
print(blockchain.chain)

[Block: {'index': 0, 'data': [], 'timestamp': 1668954286.3795156, 'previous_hash': 0, 'nonce': 0, 'hash': '9d1390a6cc109ba611856dbe989fa1172e81df25dda16c7bd049a4c21e405edf'}, Block: {'index': 1, 'data': [{'resourceType': 'Patient', 'id': 'example', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml">\n\t\t\t<table>\n\t\t\t\t<tbody>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Name</td>\n\t\t\t\t\t\t<td>Peter James1 \n              <b>Chalmers</b> (&quot;Jim&quot;)\n            </td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Address</td>\n\t\t\t\t\t\t<td>534 Erewhon, Pleasantville, Vic, 3999</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Contacts</td>\n\t\t\t\t\t\t<td>Home: unknown. Work: (03) 5555 6473</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Id</td>\n\t\t\t\t\t\t<td>MRN: 12345 (Acme Healthcare)</td>\n\t\t\t\t\t</tr>\n\t\t\t\t</tbody>\n\t\t\t</table>\n\t\t</div>'}, 'identifier': [{'use': 'usual', 'type': {'coding': [{'system': 'http://ter

In [32]:
blockchain.verify_integrity()

{'index': 0, 'data': [], 'timestamp': 1668954286.3795156, 'previous_hash': 0, 'nonce': 0, 'hash': '9d1390a6cc109ba611856dbe989fa1172e81df25dda16c7bd049a4c21e405edf'}
{'index': 1, 'data': [{'resourceType': 'Patient', 'id': 'example', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml">\n\t\t\t<table>\n\t\t\t\t<tbody>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Name</td>\n\t\t\t\t\t\t<td>Peter James1 \n              <b>Chalmers</b> (&quot;Jim&quot;)\n            </td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Address</td>\n\t\t\t\t\t\t<td>534 Erewhon, Pleasantville, Vic, 3999</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Contacts</td>\n\t\t\t\t\t\t<td>Home: unknown. Work: (03) 5555 6473</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Id</td>\n\t\t\t\t\t\t<td>MRN: 12345 (Acme Healthcare)</td>\n\t\t\t\t\t</tr>\n\t\t\t\t</tbody>\n\t\t\t</table>\n\t\t</div>'}, 'identifier': [{'use': 'usual', 'type': {'coding': [{'system': 'http://terminology.hl7.org

True

In [35]:
blockchain.chain[0].data=[]

In [36]:
blockchain.verify_integrity()

{'index': 0, 'data': [], 'timestamp': 1668954286.3795156, 'previous_hash': 0, 'nonce': 0, 'hash': '9d1390a6cc109ba611856dbe989fa1172e81df25dda16c7bd049a4c21e405edf'}
{'index': 1, 'data': [{'resourceType': 'Patient', 'id': 'example', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml">\n\t\t\t<table>\n\t\t\t\t<tbody>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Name</td>\n\t\t\t\t\t\t<td>Peter James1 \n              <b>Chalmers</b> (&quot;Jim&quot;)\n            </td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Address</td>\n\t\t\t\t\t\t<td>534 Erewhon, Pleasantville, Vic, 3999</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Contacts</td>\n\t\t\t\t\t\t<td>Home: unknown. Work: (03) 5555 6473</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Id</td>\n\t\t\t\t\t\t<td>MRN: 12345 (Acme Healthcare)</td>\n\t\t\t\t\t</tr>\n\t\t\t\t</tbody>\n\t\t\t</table>\n\t\t</div>'}, 'identifier': [{'use': 'usual', 'type': {'coding': [{'system': 'http://terminology.hl7.org

True

In [17]:
blockchain.edit_by_hash_code("0db75c70510f081ba78cf9973ba5d49d8a4e0dbb77c38fcbf4411c23ecb47246",data)

1
[Block: {'index': 2, 'data': [{'resourceType': 'Patient', 'id': 'example', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml">\n\t\t\t<table>\n\t\t\t\t<tbody>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Name</td>\n\t\t\t\t\t\t<td>Peter James \n              <b>Chalmers</b> (&quot;Jim&quot;)\n            </td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Address</td>\n\t\t\t\t\t\t<td>534 Erewhon, Pleasantville, Vic, 3999</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Contacts</td>\n\t\t\t\t\t\t<td>Home: unknown. Work: (03) 5555 6473</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Id</td>\n\t\t\t\t\t\t<td>MRN: 12345 (Acme Healthcare)</td>\n\t\t\t\t\t</tr>\n\t\t\t\t</tbody>\n\t\t\t</table>\n\t\t</div>'}, 'identifier': [{'use': 'usual', 'type': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v2-0203', 'code': 'MR'}]}, 'system': 'urn:oid:1.2.36.146.595.217.0.1', 'value': '12345', 'period': {'start': '2001-05-06'}, 'assigner': {'display

In [18]:
blockchain.chain

[Block: {'index': 0, 'data': [], 'timestamp': 1668942220.3248909, 'previous_hash': 0, 'nonce': 0, 'hash': 'e04c1f42d6f61c703f0fed1752e4f2c4de203e99cf97b457aa60ee8a1bca2fb0'},
 Block: {'index': 1, 'data': [{'resourceType': 'Patient', 'id': 'example', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml">\n\t\t\t<table>\n\t\t\t\t<tbody>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Name</td>\n\t\t\t\t\t\t<td>Peter James \n              <b>Chalmers</b> (&quot;Jim&quot;)\n            </td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Address</td>\n\t\t\t\t\t\t<td>534 Erewhon, Pleasantville, Vic, 3999</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Contacts</td>\n\t\t\t\t\t\t<td>Home: unknown. Work: (03) 5555 6473</td>\n\t\t\t\t\t</tr>\n\t\t\t\t\t<tr>\n\t\t\t\t\t\t<td>Id</td>\n\t\t\t\t\t\t<td>MRN: 12345 (Acme Healthcare)</td>\n\t\t\t\t\t</tr>\n\t\t\t\t</tbody>\n\t\t\t</table>\n\t\t</div>'}, 'identifier': [{'use': 'usual', 'type': {'coding': [{'system': 'http://ter